In [1]:
from DataPlug import DataPlug as dp
from SqueezeNet import SqueezeNet as sq

In [2]:
squeeze = sq("SqueezeNet")
dataPlug = dp("DataPlug")


In [3]:
data = dataPlug.get_reddit_dataframe()


display(dataPlug.df)

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02
53184,Hitler reacts to the market being irrational,7398,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02
53185,"Daily Discussion Thread for August 02, 2021",338,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02


In [4]:
dataPlug.get_price_datafrane()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-09-01,1.8250,1.955000,1.6925,1.912500,1.912500,92634400
1,2020-09-02,1.9500,2.012500,1.7775,1.927500,1.927500,52044400
2,2020-09-03,1.9700,2.112500,1.8100,1.955000,1.955000,57378000
3,2020-09-04,1.9450,1.980000,1.7925,1.912500,1.912500,30648000
4,2020-09-08,1.8875,2.070000,1.8700,1.925000,1.925000,39266400
...,...,...,...,...,...,...,...
653,2023-04-06,22.0000,22.670000,21.7700,22.400000,22.400000,2506900
654,2023-04-10,21.8500,22.430000,21.8200,22.370001,22.370001,2176100
655,2023-04-11,22.5700,23.170000,22.4200,22.690001,22.690001,2540000
656,2023-04-12,22.9000,23.280001,22.2600,22.260000,22.260000,2141100
